# Input/Output Parameters
**Learning more about the ```io``` section of the parameter tree - how to save the final result and intermediates into ```.ptyr``` files, and how to use live plotting with Jupyter notebooks.**

In the ```p.io``` section of the parameter tree we can specify where we would like our output to be written to. First, we can set a root directory for all further input/output operations, e.g. "/tmp"
```python
p.io.home = "/tmp"
```

PtyPy saves its results into ```.ptyr``` files which are basically [HDF5](https://docs.hdfgroup.org/hdf5/develop/) files and can therefore be inspected by any software/library that supports the HDF5 format, e.g. [h5py](https://docs.h5py.org/en/stable/). We can now specify that the main output file should be placed in a subfolder ```recon```  relative to the root directory
```python
p.io.rfile =  "recons/%(run)s_%(engine)s_%(iterations)04d.ptyr"
```
where ```%(run)s``` is dynamically replaced by the string defined in ```p.run```, ```%(engine)s``` by the name of the engine used for the reconstructions and ```%(iterations)04d``` by the total number of iterations at the end formatted as a 4-digit integer.

## Saving intermediates to file

There is also the option to auto-save intermediate states of the reconstruction at a given ```interval``` into a subfolder ```dumps``` relative to the root directory
```python
p.io.autosave = u.Param()
p.io.autosave.active = True
p.io.autosave.interval = 10
p.io.autosave.rfile = 'dumps/%(run)s_%(engine)s_%(iterations)04d.ptyr'
```
where ```%(run)s```, ```%(engine)s``` and ```%(iterations)04d``` are again dynamic labels.

<div class="alert alert-warning" markdown="1">
    <strong>Exersice</strong><br>Modify the <code>p.io.home</code> path in the example below to wherever you would like the PtyPy output to be, run the reconstruction and observe all the new files saved to <code>p.io.home</code>. Use <code>ptypy.utils.plot_recon_from_ptyr</code> to look at the MoonFlower reconstruction at different iterations. 
</div>

<div class="alert alert-success" markdown="1">
    <strong>Hint</strong><br>You can use shell commands within Jupyter Notebooks to explore the filesystem, i.e. <code>ls /tmp</code> to list all files in /tmp.
</div>

## Live-plotting

And finally, PtyPy has live-viewing capabilities that can be controlled with ```p.io.interaction``` and ```p.io.autoplot```, these are mostly useful when running a reconstruction script from a terminal with graphics enabled. They can also be used in a server-client setup, where the plotclient is running on a different machine than the reconstruction (server), see [Live visualisation](../Advanced_Topics/visualisation.ipynb) for more details. Since we are working with Jupyter notebooks for most parts of this tutorial, we turn off those features
```python
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)
```

Instead, we can use ```p.io.jupyter_autoplot``` to enable live plotting displayed within the notebook itself
```python
p.io.jupyter_autoplot = u.Param(active=True)
```
which will display the current status of the reconstruction after every contiguous block of iterations (```p.engines.engine00.numiter_contiguous```) using ```ptypy.utils.plot_recon``` with default settings. For the object subplot, the level of cropping and auto-scaling can be modified using ```p.io.jupyter_autoplot.cropping``` and ```p.io.jupyter_plotting.outlier_percentile```. 

<div class="alert alert-info" markdown="1">
<strong>Note</strong><br><code>p.io.jupyter_autoplot</code> only works with Jupyter notebooks in conjunction with <code>p.verbose_level = "interactive"</code>.
</div>

<div class="alert alert-warning" markdown="1">
    <strong>Exersice</strong><br>Modify the <code>p.io.jupyter_autoplot.cropping</code> and <code>p.io.jupyter_autoplot.outlier_percentile</code> to observe the effect these parameters have on the live reconstruction plot.  
</div>

---

In [ ]:
import ptypy
import ptypy.utils as u

p = u.Param()
p.verbose_level = "interactive"

# Run label (ID)
p.run = "moonflower"

# I/O settings
p.io = u.Param()

# Set the root path for all input/output operations
# Change this to wherever you would like 
p.io.home = "/tmp"

# Path to final .ptyr output file 
# using variables p.run, engine name and total nr. of iterations
p.io.rfile =  "recons/%(run)s_%(engine)s_%(iterations)04d.ptyr"

# Save intermediate .ptyr files (dumps) every 10 iterations
p.io.autosave = u.Param()
p.io.autosave.active = True
p.io.autosave.interval = 10
p.io.autosave.rfile = 'dumps/%(run)s_%(engine)s_%(iterations)04d.ptyr'

# Turn off traditional auto plotting
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting during the reconstruction
# that works with jupyter notebooks
p.io.jupyter_autoplot = u.Param(active=True)
p.io.jupyter_autoplot.cropping = 0.2
p.io.jupyter_autoplot.outlier_percentile = 0.1

p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0

p.engines = u.Param()
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM"
p.engines.engine00.numiter = 80
p.engines.engine00.numiter_contiguous = 5

P = ptypy.core.Ptycho(p,level=5)

In [ ]:
import ptypy.utils as u
import os
ptyr_filename = os.path.join(p.io.home, "dumps/moonflower_DM_0010.ptyr")
u.plot_recon_from_ptyr(ptyr_filename)